In [1]:
import requests
import datetime

def populate_list(number_days_window):
    populated_list = []
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live')
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live', verify=False)
    all_days = requests.get('https://api.covid19api.com/total/country/argentina')
    first=True
    for day in all_days.json():
        date_time_obj = datetime.datetime.strptime(day['Date'][:10], '%Y-%m-%d')
        if first:
            ordinal_first = date_time_obj.toordinal()
        n = date_time_obj.toordinal() - ordinal_first
        delta = 0
        fatality_rate = 0
        if n>number_days_window:
            delta = day['Confirmed'] - populated_list[n-number_days_window][1] 
        first=False
        if day['Confirmed'] > 0:
            fatality_rate = day['Deaths']/day['Confirmed']
        populated_list.append((n, day['Active'], day['Confirmed'], delta, fatality_rate, date_time_obj))
    return populated_list

In [4]:
import pandas as pd
import numpy as np

# Si fatality rate sube, se esta subtesteando, hay mas casos de lo que se cree.
# jump days cada cuantos dias sacas una foto (mirando pa atras)

def process_means(number_days_window, jump_days, total_jumps):
    my_populated_list = populate_list(number_days_window)
    means_list = []
    fatality_rate = []
    active_cases = []
    total_lapse_day = jump_days * total_jumps
    df = pd.DataFrame (my_populated_list,columns=['day-number', 'active', 'cases-count','delta', 'fatality_rate','date'])
    
    for n in range(total_lapse_day,0,-jump_days):
        try:
            mean = round(df[-(n):-(n-(jump_days))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days))]['active'].mean())
        except:
            mean = round(df[-(n):-(n-(jump_days-1))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days-1))]['active'].mean())
        means_list.append(mean)
        fatality_rate.append(df[-n:-(n-1)]['fatality_rate'].mean())
        active_cases.append(active)
    return means_list, fatality_rate, active_cases



In [6]:
total_jumps = 3
a = range(total_jumps)

#print(process_means(30,14,total_jumps))

#means_list, fatality_rate, active_cases = process_means(30,14,total_jumps)
from scipy.stats import linregress
for params in [process_means(30,14,total_jumps), process_means(14,14,total_jumps), process_means(10,14,total_jumps), process_means(7,14,total_jumps), process_means(5,14,total_jumps)]:
    means_list, fatality_rate, active_cases = params
    for result in [means_list, fatality_rate, active_cases]:
        print(result)
        print(linregress(a, result))
    print ("...----...")

[1298079, 1389986, 1498215]
LinregressResult(slope=100068.0, intercept=1295358.6666666667, rvalue=0.998893315927466, pvalue=0.029953461026538918, stderr=4711.755546856499)
[0.027113823295303794, 0.027209919430055626, 0.026928903664589072]
LinregressResult(slope=-9.245981535736107e-05, intercept=0.027176675278673523, rvalue=-0.6473492547820558, pvalue=0.5517550677621113, stderr=0.00010886282855284805)
[128735, 130997, 146477]
LinregressResult(slope=8871.0, intercept=126532.0, rvalue=0.9186249156838107, pvalue=0.2586013919587784, stderr=3815.707929074236)
...----...
[1311938, 1404275, 1499274]
LinregressResult(slope=93668.0, intercept=1311494.3333333333, rvalue=0.9999663488086744, pvalue=0.005222717805116478, stderr=768.4532082889356)
[0.027113823295303794, 0.027209919430055626, 0.026928903664589072]
LinregressResult(slope=-9.245981535736107e-05, intercept=0.027176675278673523, rvalue=-0.6473492547820558, pvalue=0.5517550677621113, stderr=0.00010886282855284805)
[128735, 130997, 146477]


# Algunas reglas

Si fatality_rate > fatality_rate_mean:
    Yellow Warning Casos pueden ser mayor de lo reflejado pues pueden que estén subtesteando.

Si fatality_rate > fatality_rate_mean y active_cases y means_list en crecimiento:

    Rojo Warning 

means_list, fatality_rate, active_cases


In [4]:
from random import choice
from experta import *


class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
    def cautious(self, light):
        print("Be cautious because light is", light["color"])


engine1 = RobotCrossStreet()
engine1.reset()
engine1.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine1.run()


Be cautious because light is blinking-yellow
